In [ ]:
import os
from glob import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../../')
from base_ml.utils import MLOutputHandler, ClassifierOutputHandler
from base_ml import utils

%load_ext autoreload
%autoreload 2

In [ ]:
# Helper function
def get_model_names(path_list):
    res_list = []
    for i in path_list:
        cur_name = i.split('/')[-1]
        cur_name = cur_name.replace('output_pred_', '').replace('_.pkl', '')
        res_list.append(cur_name)
    return res_list

### 1. Plot classification output

In [ ]:
# Specify path of output pickle files from
pkl_list = [
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_LinearDiscriminantAnalysis_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_LogisticRegression_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_SVC_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_GaussianNB_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_GaussianProcessClassifier_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_KNeighborsClassifier_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_DecisionTreeClassifier_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_RandomForestClassifier_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_AdaBoostClassifier_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_MLPClassifier_.pkl',
    '/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_MGMCCheby_100.0remain.pkl',
]

all_df = []
cm_fig_list1 = []
# model_name = ['LR', 'RF', 'MLP', 'MaskedMLP', 'GCN', 'MaskedGCN', 'InceptionGCN', 'MaskedInceptionGCN', 'MGMC (ChebConv)']
# pkl_list = glob('/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/output_pred_*.pkl')
model_name = get_model_names(pkl_list)
model_name = [x.replace("Cheby_100.0remain.pkl", "") if "Cheby_100.0remain" in x else x for x in model_name]

for k,v in enumerate(pkl_list):
    print(model_name[k], v)
    with open(v, 'rb') as fpath:
        y_list = pickle.load(fpath)
        cur_df = ClassifierOutputHandler.to_dataframe(y_list, model_name[k], average='macro')
        all_df.append(cur_df)
        cur_fig = ClassifierOutputHandler.to_confusion_matrix(y_list, col_name=model_name[k], 
                                                    class_names=['NC', 'MCI', 'AD'])
        cm_fig_list1.append(cur_fig)
res_df = pd.concat(all_df, ignore_index=True)

# Plot boxplots
out_fig = ClassifierOutputHandler.pretty_df_to_boxplots(res_df, y_limit=(0.0, 1.05))


### 2. Save outputs

In [ ]:
# Summary of classification output in dataframe format
save_table_t1 = res_df[['value', 'estimator', 'variable']].groupby(['estimator', 'variable']).describe().round(3)
save_table_t1

# Uncomment command below to save dataframe to specified location
# save_table_t1.to_excel('/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/dz_clf_metrics_df_describe.xlsx')

In [ ]:
# Save confusion matrices
output_dir = "/workspace/base-ml/examples/outputs/dizzyreg1/exp_008/"
utils.save_fig_list(output_dir, cm_fig_list1)